<a href="https://colab.research.google.com/github/AleksanderHal/arpdata/blob/main/PROJEKT_KONCOWY2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#IMPORT

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing

In [2]:
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 350)

df = pd.read_csv(r"/content/flights[1].csv")
airports = pd.read_csv(r"/content/airports[1].csv")
airlines = pd.read_csv(r"/content/airlines[1].csv")
airlines = airlines.rename(columns={'AIRLINE': 'AL_FULLNAME', 'IATA_CODE': 'AIRLINE'})

#convert column names to lowercase
df.columns = df.columns.str.lower()
airlines.columns = airlines.columns.str.lower()
airports.columns = airports.columns.str.lower()

FileNotFoundError: ignored

In [ ]:
import pandas as pd

# Merge airlines and flights data to get full names of airlines
df = df.merge(airlines, how='left', on='airline')

# Merge origin airport data onto the main df
merged_df = df.merge(airports, left_on='origin_airport', right_on='iata_code', how='left')

# Rename columns
column_rename_mapping = {
    'airport': 'origin_airport_name',
    'city': 'origin_city',
    'state': 'origin_state',
    'country': 'origin_country',
    'latitude': 'origin_latitude',
    'longitude': 'origin_longitude'
}
merged_df = merged_df.rename(columns=column_rename_mapping)

# Drop the 'iata_code' column
merged_df = merged_df.drop('iata_code', axis=1)






In [ ]:
# Drop rows with NaN in the 'departure_delay' and 'arrival_delay' columns for cancelation prediction
#merged_df.dropna(subset=['departure_delay', 'arrival_delay', 'cancelled'], inplace=True)


In [ ]:

# Merge destination airport data onto the main df, rename columns, and drop 'iata_code'
df = pd.merge(merged_df, airports, left_on='destination_airport', right_on='iata_code', how='left')\
    .rename(columns={
        'airport': 'destination_airport_name',
        'city': 'destination_city',
        'state': 'destination_state',
        'country': 'destination_country',
        'latitude': 'destination_latitude',
        'longitude': 'destination_longitude'
    })\
    .drop('iata_code', axis=1)

In [ ]:
result = pd.DataFrame({
    'Data Types': df.dtypes,
    'Null Values': df.isnull().sum(),
    'Null Percent': (df.isnull().mean() * 100),
    'Unique Values': df.nunique()
}).T

# Display the result
result

In [ ]:
#drop columns with little variability/not useful in prediciton/visualization or redundant
df = df.drop(columns = ['flight_number', 'tail_number', 'taxi_out',
                   'wheels_off', 'air_time', 'taxi_in', 'diverted',
                  'cancellation_reason', 'air_system_delay', 'security_delay',
                   'airline_delay', 'late_aircraft_delay', 'weather_delay', 'destination_country',
                       'origin_country'])
df.head()
df

In [ ]:
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
df['week_number'] = df['date'].dt.strftime('%U')
df['sched_hour'] = df['departure_time'] // 100  # Assuming you want to extract the hour from 'departure_time'
df['dow_name'] = df['date'].dt.strftime('%A')

In [ ]:
def season_cat(x):
    season_mapping = {
        12: 'winter',
        1: 'winter',
        2: 'winter',
        3: 'spring',
        4: 'spring',
        5: 'spring',
        6: 'summer',
        7: 'summer',
        8: 'summer'
    }
    return season_mapping.get(x, 'autumn')

In [ ]:
# Create a new column for the season
df['season'] = df['month'].apply(season_cat)

# Drop the 'year' column
df.drop(columns='year', inplace=True)

In [ ]:
# Find unmatched origin airports
unmatched_origin_airports = set(df['origin_airport']) - set(airports['iata_code'])

# Find unmatched destination airports
unmatched_destination_airports = set(df['destination_airport']) - set(airports['iata_code'])

# Calculate the number of unmatched origin airports
num_unmatched_origin_airports = len(unmatched_origin_airports)

num_unmatched_origin_airports  # This will give you the count of unmatched origin airports

In [ ]:
from datetime import datetime
import pandas as pd

# Function to convert float time to string time format
def convert_float_time(float_time):
    if pd.notna(float_time):
        float_time = str(int(float_time)).zfill(4)  # Ensure 4 characters
        hour = int(float_time[:2])  # Extract the hour
        minute = int(float_time[2:])  # Extract the minutes
        if hour >= 24:
            hour = hour % 24
        return f"{hour:02d}:{minute:02d}:00"
    return None

# Apply the conversion function to the time-related columns
time_columns = ['arrival_time', 'departure_time', 'scheduled_departure']
for col in time_columns:
    df[col] = df[col].apply(convert_float_time)

# Combine date and scheduled departure columns into a single datetime column
df['scheduled_departure_datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['scheduled_departure'], format='%Y%m%d %H:%M:%S')

# Get the hour from the scheduled departure time and store it in a new column
df['sched_hour'] = df['scheduled_departure_datetime'].dt.strftime('%H')

#EDA


In [ ]:

# Oblicz macierz korelacji
#correlation_matrix = df[['departure_delay', 'arrival_delay', 'cancellation']].corr()

# Wyświetl macierz korelacji
#print(correlation_matrix)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Get the count of all unique airlines
airline_counts = df['al_fullname'].value_counts().sort_values(ascending=False)

# Set the style for the plot
sns.set(style="whitegrid")

# Create a bar chart using seaborn
plt.figure(figsize=(8, 6))
sns.barplot(x=airline_counts.values, y=airline_counts.index, palette="colorblind")

# Set plot title and axis labels
plt.title("Count of Flights of all Airlines")
plt.xlabel("Count")
plt.ylabel("Airline")

# Display the plot
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate the total distance flown by each airline
airline_sum_distance = df.groupby('al_fullname')['distance'].sum().sort_values(ascending=False)

# Set the style for the plot
sns.set(style="whitegrid")

# Create a bar plot using seaborn
plt.figure(figsize=(8, 6))
sns.barplot(x=airline_sum_distance.values, y=airline_sum_distance.index, palette="colorblind")

# Set plot title and axis labels
plt.title("Total Sum of Distance Flown by Airline")
plt.xlabel("Total Distance (in miles)")
plt.ylabel("Airline")

# Display the plot
plt.show()

In [ ]:

df['distance'].plot(kind = 'hist')

In [ ]:
# Function that assigns 'short,' 'medium,' or 'long' based on distance
def dist_cat(x):
    if x < 500:
        return 'short'
    elif x <= 1000:
        return 'medium'
    return 'long'

# Create 'dist_type' column and count the number of short, medium, and long flights
df['dist_type'] = df['distance'].apply(dist_cat)
dist_type_counts = df['dist_type'].value_counts()

# Print the count of flights in each category
print(dist_type_counts)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.countplot(data=df, y='al_fullname', hue='dist_type', palette="colorblind")
plt.xlabel("Count")
plt.ylabel("Airline Full Name")
plt.title("Count of Dist Types by Airline")
plt.legend(title='Distribution Type')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.stripplot(y="al_fullname", x="cancelled", data=df, linewidth = 2,  jitter=True)

In [ ]:
#15 destination airports with the largest average number of flights cancelled
df.groupby('destination_airport')['cancelled'].mean().sort_values().nlargest(15)

Does airline performance change over the months of the year? In other words, is one airline always the worst performing, or is there variability?

Which routes have the highest probability of being canceled?

In [ ]:
df['cancelled'].value_counts(normalize=True) * 100

In [ ]:
sns.countplot(x='cancelled', data = df)
plt.show()

In [ ]:
df.groupby(['al_fullname'])['cancelled'].mean().sort_values(ascending = False).reset_index()

In [ ]:
# Group the DataFrame by date, calculate the sum of cancellations, and sort the results
cancellation_counts = df.groupby(['date'])['cancelled'].sum().sort_values(ascending=False)

# Create a figure with a specified size
plt.figure(figsize=(10, 6))

# Plot the cancellation counts
cancellation_counts.plot()

# Set plot title and axis labels
plt.title('Cancellation Count 2015')
plt.xlabel('Date')
plt.ylabel('Cancellation Count')

# Display the plot
plt.show()

In [ ]:
# Count cancellations and flights for each airline
df_cancelled = df.groupby('al_fullname')['cancelled'].agg(total_cancelled='sum', total_flights='count')
df_cancelled['cancellation_rate'] = df_cancelled['total_cancelled'] / df_cancelled['total_flights']

# Plot a scatter plot for the top 7 airlines with the highest total flights
top_airlines = df_cancelled.nlargest(7, 'total_flights')
scatter_plot = sns.scatterplot(data=top_airlines.reset_index(), x='total_flights', y='cancellation_rate', hue='al_fullname', marker='o', s=100)
plt.xlabel('Total Flights')
plt.ylabel('Cancellation Rate')
plt.title('Cancellation Rate vs. Total Flights for Top 7 Airlines')

# Move the legend outside the graph
plt.legend(title='Airline', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()





In [ ]:
# Get the average cancellations by day of the week
dow_canc = df.groupby(['dow_name'])['cancelled'].mean().reset_index()
dow_canc['cancellation_mean'] = dow_canc['cancelled'] * 100
dow_canc.drop(columns=['cancelled'], inplace=True)

# Get the count of flights by day of the week
dow_count = df.groupby(['dow_name']).size().reset_index()
dow_count.rename(columns={0: 'flight_count'}, inplace=True)

# Merge the dataframes and sort by flight count
canc_df = pd.merge(dow_canc, dow_count, on='dow_name').sort_values(by='flight_count')

# Display the resulting dataframe
canc_df

Forecasting the count of cancelled American Airlines flights.

In [ ]:
#subset df to include only american airline flights
al_df = df[(df['airline'] == 'AA')]

#drop observations from october
al_df[al_df['month'] != 10]

#subset dataframe to predictive columns
#removing columns where data isn't available(e.g. departure delay isn't known for cancelled flight)
relevant_cols = ['cancelled','month','day_of_week','distance','scheduled_time', 'sched_hour']
al_df = al_df[relevant_cols]

In [ ]:
#add day of week, route, hour, month dummy variables
day_dummys = pd.get_dummies(al_df['day_of_week'], prefix = 'day')
month_dummys = pd.get_dummies(al_df['month'], prefix = 'month')
hour_dummys = pd.get_dummies(al_df['sched_hour'], prefix = 'hour')

In [ ]:
#concat dummy vars to dataframe
al_df = pd.concat([al_df, day_dummys, month_dummys, hour_dummys], axis = 1)
al_df = al_df.drop(columns = ['month','day_of_week', 'sched_hour'])

In [ ]:
al_df.head()

In [ ]:
#split data into features + target
X = al_df.drop('cancelled', 1)
y = al_df.cancelled

#MODEL

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Split the dataset into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.impute import SimpleImputer

# Handle missing values with simple imputation
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)


# Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Display evaluation results
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)